### Prediction ###

In [2]:
from transformers import AutoModelForSequenceClassification, DistilBertTokenizer
import torch
import numpy as np
import torch.nn.functional as F

model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model")
tokenizer = DistilBertTokenizer.from_pretrained("./fine_tuned_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


def tokenize_input(input_text):
    return tokenizer(
        input_text,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        max_length=128,
    )


def map_output(value: np.int64) -> str:
    mapping = {
        0: "negative",
        1: "positive",
    }
    return mapping.get(value, "???")


def predict(input_texts):
    model.eval()
    predictions = []

    for text in input_texts:
        inputs = tokenize_input(text)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        probs = F.softmax(logits, dim=1)
        predicted_class = np.argmax(probs.cpu().numpy(), axis=1)

        prediction_label = map_output(predicted_class[0])
        prediction_confidence = probs[0, predicted_class].item() * 100

        predictions.append((prediction_label, prediction_confidence))

    return predictions


new_texts = [
    "This movie was amazing! Don't you think?",
    "I hated this product. It's terrible.",
    "The book was okay, not great but not bad either.",
    "Neutral",
    "Everything",
]

predictions = predict(new_texts)

for text, (prediction, confidence) in zip(new_texts, predictions):
    print(f"text: {text}\nprediction: {prediction}\nconfidence: {confidence:.2f}%\n")


text: This movie was amazing! Don't you think?
prediction: positive
confidence: 99.99%

text: I hated this product. It's terrible.
prediction: negative
confidence: 99.98%

text: The book was okay, not great but not bad either.
prediction: positive
confidence: 99.95%

text: Neutral
prediction: negative
confidence: 99.92%

text: Everything
prediction: positive
confidence: 99.97%

